# Image Classification using Logistic Regression and K-Means

This notebook demonstrates image classification on a facial expression dataset using:
- **Logistic Regression** (supervised classifier)
- **K-Means** (unsupervised clustering, adapted for classification)

We will work with up to 5 emotion classes from the dataset.


## 1. Importing Required Libraries


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import cv2
import warnings
warnings.filterwarnings('ignore')


## 2. Loading the Dataset


In [2]:
# Load training data
print("Loading training data...")
train_df = pd.read_csv('Data/train.csv')

# Check the structure
print(f"Training data shape: {train_df.shape}")
print(f"Columns: {train_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(train_df.head())

# Check unique emotion classes
unique_emotions = sorted(train_df['emotion'].unique())
print(f"\nUnique emotion classes: {unique_emotions}")
print(f"Number of classes: {len(unique_emotions)}")


Loading training data...
Training data shape: (28709, 2)
Columns: ['emotion', 'pixels']

First few rows:
   emotion                                             pixels
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1        0  151 150 147 155 148 133 111 140 170 174 182 15...
2        2  231 212 156 164 174 138 161 173 182 200 106 38...
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...

Unique emotion classes: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]
Number of classes: 7


## 3. Limiting to 5 Classes (if needed)


In [ ]:
# Limit to maximum 5 classes
max_classes = 5
if len(unique_emotions) > max_classes:
    print(f"Limiting dataset to first {max_classes} classes...")
    train_df = train_df[train_df['emotion'].isin(unique_emotions[:max_classes])]
    print(f"New dataset shape: {train_df.shape}")
    unique_emotions = sorted(train_df['emotion'].unique())
    print(f"Classes used: {unique_emotions}")
else:
    print(f"Dataset already has {len(unique_emotions)} classes (≤ {max_classes})")


## 4. Converting Pixel Strings to Images


In [ ]:
def pixels_to_image(pixel_string, img_size=(48, 48)):
    """
    Convert space-separated pixel string to numpy array image.
    
    Parameters:
    - pixel_string: String of space-separated pixel values
    - img_size: Tuple (height, width) for image dimensions
    
    Returns:
    - Image as numpy array of shape (height, width)
    """
    pixels = np.array(pixel_string.split(), dtype='float32')
    image = pixels.reshape(img_size)
    return image

# Convert pixel strings to images
print("Converting pixel strings to images...")
images = np.array([pixels_to_image(pixel_str) for pixel_str in train_df['pixels']])
labels = train_df['emotion'].values

print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Pixel value range: [{images.min():.1f}, {images.max():.1f}]")


## 5. Visualizing Sample Images


In [ ]:
# Display sample images from each class
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

fig, axes = plt.subplots(1, len(unique_emotions), figsize=(15, 3))
if len(unique_emotions) == 1:
    axes = [axes]

for idx, emotion in enumerate(unique_emotions):
    # Find first image of this emotion
    emotion_idx = np.where(labels == emotion)[0][0]
    axes[idx].imshow(images[emotion_idx], cmap='gray')
    axes[idx].set_title(f"Class {emotion}\n{emotion_labels.get(emotion, 'Unknown')}")
    axes[idx].axis('off')

plt.suptitle('Sample Images from Each Class', fontsize=14)
plt.tight_layout()
plt.show()


## 6. Image Preprocessing

We'll apply several preprocessing techniques from the tutorial:
- Normalization
- Grayscale conversion (already grayscale)
- Optional: Histogram equalization
- Optional: Gaussian blur for noise reduction


In [ ]:
def preprocess_images(images, apply_hist_eq=False, apply_gaussian=False):
    """
    Preprocess images: normalize and optionally apply histogram equalization and Gaussian blur.
    
    Parameters:
    - images: Array of images (n_samples, height, width)
    - apply_hist_eq: Whether to apply histogram equalization
    - apply_gaussian: Whether to apply Gaussian blur
    
    Returns:
    - Preprocessed images
    """
    processed_images = images.copy().astype('float32')
    
    # Normalize to [0, 1]
    for i in range(len(processed_images)):
        img = processed_images[i]
        img_min, img_max = img.min(), img.max()
        if img_max > img_min:
            processed_images[i] = (img - img_min) / (img_max - img_min)
    
    # Convert to uint8 for OpenCV operations
    processed_images_uint8 = (processed_images * 255).astype('uint8')
    
    # Apply histogram equalization if requested
    if apply_hist_eq:
        for i in range(len(processed_images_uint8)):
            processed_images_uint8[i] = cv2.equalizeHist(processed_images_uint8[i])
    
    # Apply Gaussian blur if requested
    if apply_gaussian:
        for i in range(len(processed_images_uint8)):
            processed_images_uint8[i] = cv2.GaussianBlur(processed_images_uint8[i], (3, 3), 0)
    
    # Convert back to [0, 1] range
    processed_images = processed_images_uint8.astype('float32') / 255.0
    
    return processed_images

# Apply preprocessing
print("Preprocessing images...")
processed_images = preprocess_images(images, apply_hist_eq=False, apply_gaussian=False)
print(f"Preprocessed images shape: {processed_images.shape}")
print(f"Preprocessed pixel value range: [{processed_images.min():.3f}, {processed_images.max():.3f}]")


## 7. Flattening Images for Classification

Both Logistic Regression and K-Means require 2D input (samples × features).


In [ ]:
# Flatten images to feature vectors
n_samples, height, width = processed_images.shape
X = processed_images.reshape(n_samples, -1)  # Flatten to (n_samples, height*width)
y = labels

print(f"Feature matrix shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of features per image: {X.shape[1]}")


## 8. Train-Test Split


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nClass distribution in training set:")
unique, counts = np.unique(y_train, return_counts=True)
for emotion, count in zip(unique, counts):
    print(f"  Class {emotion}: {count} samples")


## 9. Feature Scaling

Standardizing features is important for Logistic Regression and can help K-Means converge faster.


In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Scaled training features - Mean: {X_train_scaled.mean():.4f}, Std: {X_train_scaled.std():.4f}")
print(f"Scaled test features - Mean: {X_test_scaled.mean():.4f}, Std: {X_test_scaled.std():.4f}")


## 10. Logistic Regression Classifier
in image data there is 3 CSV I WANT TO CREATE. Logistic regression & kmeans as classifiers on an image dataset (5 classes at maximum). IN JUPYTER NOTEBOOK 

HERE IS THE IMAGE PREPROCCESING TURTORIALS TA GAVE 

##%%
#importing required Libraries
import numpy as np
import tensorflow
import keras
import os
import glob
from skimage import io
import skimage
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
##%% md
# Image Preprocessing in Python (Lecture Notebook)

This notebook introduces fundamental image preprocessing techniques using **Python**, **OpenCV**, **scikit-image**, and **Keras**.

We will cover:

- Loading and visualizing images
- RGB channels and grayscale conversion
- Normalization
- Data augmentation (flipping and rotation)
- Contrast enhancement (Histogram Equalization & CLAHE)
- Smoothing filters (mean, Gaussian, median)
- Edge detection (Sobel, Laplacian)
- Sharpening with custom kernels
- Thresholding and basic image segmentation

##%% md
## 1. Loading and Visualizing the Original Image

In this step we:

1. Load an image from disk using `skimage.io.imread`.
2. Visualize it using `matplotlib`.

Why this step is important:

- It lets us inspect the **raw image**.
- We can observe:
- Resolution (width × height)
- Colors and lighting
- Any visible noise or blur
- This serves as a **baseline** before applying preprocessing.

> Make sure `fruits.png` is in the same folder as your notebook, or provide the full path.

##%%
import os
# accessing an image file from the dataset classes
image = io.imread('fruits.png')

# plotting the original image
i, (im1) = plt.subplots(1)
i.set_figwidth(15)
im1.imshow(image)
##%% md
## 2. Visualizing the Original Image and Its RGB Color Channels

Color images are stored as 3D arrays: **(height, width, channels)**.

- Channel 0 → Red
- Channel 1 → Green
- Channel 2 → Blue

In this section we:

- Show the original image
- Show each channel separately (R, G, B)

Why this helps:

- Understand which colors dominate the image.
- See how information is distributed across channels.
- Useful before performing color-based processing (e.g. segmentation, enhancement).

##%%
# plotting the original image and the RGB channels

i, (im1, im2, im3, im4) = plt.subplots(1, 4, sharey=True) # sharey -> All subplots will share the same Y-axis.
i.set_figwidth(20)
print(image.shape)
im1.imshow(image) #Original image
im2.imshow(image[:, : , 0]) #Red
im3.imshow(image[:, : , 1]) #Green
im4.imshow(image[:, : , 2]) #Blue
i.suptitle('Original & RGB image channels')
##%% md
The RGB image is converted into a **grayscale** representation using `skimage.color.rgb2gray()`.
A grayscale image reduces the three color channels (Red, Green, and Blue) into a single intensity channel by applying a weighted sum that reflects human visual perception. This simplification is commonly used in many preprocessing tasks, such as edge detection, thresholding, and filtering, where color information is not required.

The resulting grayscale image is then displayed using matplotlib with the 'gray' colormap to ensure proper visualization of intensity variations.
##%% md
## 3. Converting RGB Image to Grayscale

We convert the RGB image into a **single-channel grayscale** image using:

```python
skimage.color.rgb2gray(image) ---> it is automatically normalizes the grayscale image to the range [0, 1].

##%%
gray_image = skimage.color.rgb2gray(image)
plt.imshow(gray_image, cmap = 'gray')
##%% md
## 4. Normalization (Min–Max Scaling)

We normalize the grayscale image to the range **[0, 1]** using:

$$
\text{norm}(I) = \frac{I - I_{\min}}{I_{\max} - I_{\min}}
$$

Why normalize?

- Standardizes pixel values.
- Improves numerical stability for ML/DL models.
- Helps networks train faster and more reliably.
- Makes images from different sources more comparable.

We will compute the normalized image and display it.

##%%
norm_image = (gray_image - np.min(gray_image)) / (np.max(gray_image) - np.min(gray_image))
plt.imshow(norm_image)
##%% md
## 5. Installing `keras_preprocessing` (For Data Augmentation)

To perform geometric transformations (data augmentation) in Keras, we use:

- `ImageDataGenerator` from `keras_preprocessing.image`

If the package is not already installed, we install it using `pip` inside the notebook.

##%%
#!pip install keras_preprocessing
##%% md
## 6. Preparing Image Batch for Data Augmentation

`ImageDataGenerator` expects input as a batch of images with shape:

- `(batch_size, height, width, channels)`

We will:

1. Convert our image to `float32`.
2. Add a new axis to create a batch of size 1 using `np.expand_dims`.

This will be used for flipping and rotation examples.

##%%
from numpy import expand_dims
print(image.shape)
# Ensure we use the RGB image for augmentation
samples = expand_dims(image.astype('float32'), axis=0) # shape: (1, H, W, 3)


##%%
print("Batch shape:", samples.shape)

##%% md
# Geometric Transformations
In this step, we apply **geometric transformations** to the images using the `ImageDataGenerator` class from `Keras`. Geometric transformations are a form of data augmentation.

We use `ImageDataGenerator` to apply:

- `horizontal_flip=True` → flip left–right
- `vertical_flip=True` → flip top–bottom

Why do this?

- Data augmentation: increases dataset size without collecting new images.
- Makes models more robust to orientation changes.
- Helps prevent overfitting.

Explain:

* Creating an ImageDataGenerator instance with the desired transformations.

* Generating batches of images using the `.flow()` method, which produces transformed images on the fly.

* Iterating through the generated images and converting them to unsigned integers (`uint8`) for proper visualization.

* Plotting the transformed images side by side to observe the effects of horizontal and vertical flips.

We will:

1. Create an `ImageDataGenerator` with flipping options.
2. Generate 3 augmented images.
3. Display them side by side.
##%%
from keras_preprocessing.image import ImageDataGenerator

# ImageDataGenerator for flipping
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

# Create an iterator
it = datagen.flow(samples, batch_size=1) #batch_size=1 ->Each time you call the iterator, it returns only 1 augmented image.

# Plot some flipped images
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

for i in range(3):
# Get a batch (1 image), take first image
batch = next(it)[0].astype('uint8')
axes[i].imshow(batch)
axes[i].set_title(f"Flip Sample {i+1}")
axes[i].axis("off")

plt.suptitle("Horizontal & Vertical Flips")
plt.show()

##%% md
## 8. Geometric Transformations – Rotation


Apply `random rotation transformations` to the images as part of data augmentation. The `rotation_range` parameter specifies the maximum rotation angle (in degrees) for randomly rotating images. Here, `rotation_range=40` allows images to rotate within ±40 degrees.

**Why use rotation?**
- Simulates different orientations.
- Helps models generalize better when objects are rotated in real-world data.
##%%
# ImageDataGenerator for rotation
# Fills empty pixels with the value of the nearest pixel
# The image will be randomly rotated between –40° and +40°.
datagen = ImageDataGenerator(rotation_range=40, fill_mode='nearest') #

# Create an iterator
it = datagen.flow(samples, batch_size=1)

# Plot some flipped images
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

for i in range(3):
# Get a batch (1 image), take first image
batch = next(it)[0].astype('uint8')
axes[i].imshow(batch)
axes[i].set_title(f"Rotation Sample {i+1}")
axes[i].axis("off")

plt.suptitle("Random Rotations (±40°)")
plt.show()
##%% md
## 9. Histogram Equalization for Contrast Enhancement (زيادة التباين + تحسين الوضوح)

We now switch to **OpenCV** (`cv2`) for some operations.

### Goal:
- Improve image contrast using **histogram equalization**.

Steps:
1. Load the image in **grayscale**.
2. Apply `cv2.equalizeHist`.
3. Compare original vs equalized images.

Histogram equalization:
- Spreads pixel intensities (بتوزّع قيم الإضاءة) across the full range `[0, 255]`.
- Makes dark regions brighter and bright regions clearer. (إظهار التفاصيل المخفية)
- Helpful when the image looks too dark or too washed out. (مفيد في الصور ذات الإضاءة السيئة)

##%%
import cv2
import matplotlib.pyplot as plt

# Load grayscale image
img = cv2.imread("fruits.png", 0) # 0 → grayscale

# Histogram equalization
equalized = cv2.equalizeHist(img)

# Plot results
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Original Grayscale Image")
plt.imshow(img, cmap="gray")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Equalized Image")
plt.imshow(equalized, cmap="gray")
plt.axis("off")

plt.show()

##%% md
## 10. Visualizing Histograms Before and After Equalization

To better understand the effect of **histogram equalization**, we plot the pixel intensity histograms of the original and equalized images.

* `The original histogram` shows the distribution of pixel values in the grayscale image. In many cases, the values are concentrated in a narrow range, which can make the image appear dark or washed out.

* `The equalized histogram` illustrates how the pixel intensities have been redistributed across the full range (0–255). This spreading of values increases the contrast and highlights previously hidden details.

By comparing these histograms side by side, it becomes evident that histogram equalization effectively enhances image contrast while preserving the overall structure of the image. Visual inspection alongside histogram analysis provides a clear and quantitative understanding of the preprocessing step’s impact.
##%%
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.title("Original Histogram")
plt.hist(img.ravel(), 256, [0, 256])

plt.subplot(1, 2, 2)
plt.title("Equalized Histogram")
plt.hist(equalized.ravel(), 256, [0, 256])

plt.show()

##%% md
## 11. Contrast Enhancement Using CLAHE

**CLAHE** = Contrast Limited Adaptive Histogram Equalization.
- used to improve the contrast of a color image

Differences vs normal histogram equalization:

- Works on **small tiles** instead of whole image. 
- Limits contrast amplification (avoids over-enhancing noise). ( يحد من تضخيم التباين (يتجنب التشويش المفرط).)
- Very useful for images with **uneven lighting**. (الإضاءة غير المتساوية)
- A small amount of noise may appear due to processing each tile separately.

Steps:

1. Convert BGR image to LAB color space. (LAB separates lightness (L) from color (A and B).)
2. Apply CLAHE on the L (lightness) channel.
3. Merge channels and convert back to BGR/RGB.
4. Compare before and after.

**LAB channels:**
- L = Lightness (0–255)
- A = Green–Red scale
- B = Blue–Yellow scale
##%% md
**clipLimit=3.0**
- Prevents over-amplifying noise
- Higher value → stronger contrast

**tileGridSize=(8, 8)**
- Image is divided into 8×8 regions
- Contrast enhancement is applied locally
- Good for images with uneven lighting
##%%
# Load image (BGR)
img_color = cv2.imread("fruits.png")

# Convert to LAB color space
lab = cv2.cvtColor(img_color, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(lab)

# Apply CLAHE on L-channel (lightness)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
cl = clahe.apply(l)

# Merge channels and convert back to BGR
lab_clahe = cv2.merge((cl, a, b))
final = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# Show Results
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("After CLAHE")
plt.imshow(cv2.cvtColor(final, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.show()

##%% md
## 12. Low-Pass Filters (Smoothing / Blurring)

In this step, we apply a **Mean Filter** to the grayscale image. The mean filter is a type of low-pass filter that **smooths** the image by reducing intensity variations between neighboring pixels. **It works by replacing each pixel value with the average of its surrounding pixels defined by a kernel** (in this case, a 3×3 window).


We will apply:

1. **Mean Filter** (يأخذ متوسط قيم البكسلات داخل نافذة (Kernel).)
2. **Gaussian Blur**
3. **Median Filter** 

Why smoothing?

- Reduces noise.
- Softens edges.
- Often used before edge detection or segmentation.
##%%
# Reload grayscale image for filtering
img = cv2.imread("fruits.png", 0)

# Mean (average) filter with 3x3 kernel
mean = cv2.blur(img, (3, 3))

plt.imshow(mean, cmap="gray")
plt.title("Mean Filter (3x3)")
plt.axis("off")
plt.show()

##%%
# Gaussian blur with 5x5 kernel, sigma = 0 (auto)
gaussian = cv2.GaussianBlur(img, (5, 5), 0)

plt.imshow(gaussian, cmap="gray")
plt.title("Gaussian Blur (5x5)")
plt.axis("off")
plt.show()

##%%
median = cv2.medianBlur(img, 5) # 5x5 neighborhood

plt.imshow(median, cmap="gray")
plt.title("Median Filter (5x5)")
plt.axis("off")
plt.show()

##%% md
## 13. High-Pass Filters (Edge Detection & Sharpening)

Edge Detection Using **Sobel Filter**

The Sobel operator computes the gradient of pixel intensities(تدرج شدة البكسل) in both horizontal (X) and vertical (Y) directions, highlighting regions with significant intensity changes.

* sobel(x) detects vertical edges by calculating horizontal intensity gradients.

* sobel(y) detects horizontal edges by calculating vertical intensity gradients.

Edge detection is a fundamental preprocessing step in computer vision, used to identify object boundaries, enhance features for segmentation, and extract structural information from images. Visualizing both X and Y gradients separately allows us to analyze the directionality and strength of edges in the image.
##%% md
**Understanding dx and dy**

**- dx = 1, dy = 0 → detect changes along X-axis**
- Finds vertical edges
- (Because vertical edges change in X direction)

**- dx = 0, dy = 1 → detect changes along Y-axis**
- Finds horizontal edges
- (Because horizontal edges change in Y direction)
##%%
sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3) # depth,1->SobelX, 0->SobelY
sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3) 

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(sobelx, cmap="gray")
plt.title("Sobel X (Vertical Edges)")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(sobely, cmap="gray")
plt.title("Sobel Y (Horizontal Edges)")
plt.axis("off")

plt.show()

##%% md
### 🔍 Laplacian Filter (Simplified Explanation)

The **Laplacian filter** is a high-pass filter used to detect edges in an image.

- It responds to **sharp changes in intensity**.
- Unlike Sobel (which finds horizontal or vertical edges separately), 
**Laplacian detects edges in all directions at once**.
- This makes it useful for highlighting **fine details** and **object boundaries**.

The resulting image shows bright edges and suppresses smooth, low-detail areas.

##%%
laplacian = cv2.Laplacian(img, cv2.CV_64F)

plt.imshow(laplacian, cmap="gray")
plt.title("Laplacian (All Edges)")
plt.axis("off")
plt.show()

##%% md
### Sharpening Filter (Custom Kernel)

In this step, we apply a sharpening filter to enhance the details and edges of the grayscale image. The filter is implemented using a convolution kernel that emphasizes the central pixel relative to its neighbors:

$$
\begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0
\end{bmatrix}
$$
- The center pixel is multiplied by 5.
- Neighboring pixels are subtracted.
- This increases contrast at edges → image looks sharper.
- This is a **balanced** sharpening kernel.

This kernel increases the contrast between a pixel and its surrounding pixels, effectively highlighting edges and fine structures while maintaining the overall brightness of the image. Sharpening is a common preprocessing technique to improve the visibility of important features, which can be useful in tasks such as object recognition, segmentation, and feature extraction.
##%%
kernel = np.array([[0,-1,0],
[-1,5,-1],
[0,-1,0]])

sharpened = cv2.filter2D(img, -1, kernel)

plt.imshow(sharpened, cmap="gray")
plt.title("Sharpened Image")
plt.axis("off")
plt.show()

##%% md
## 14. Segmentation with Thresholding

### 🌓 Binary Thresholding

In this step, we apply binary thresholding to the grayscale image. Thresholding is a fundamental technique in image segmentation.

Binary thresholding separates the image into **two groups of pixels**: 
- **Foreground (white)** 
- **Background (black)** 

We use `cv2.threshold()` with a threshold value of **127**:

- Pixels with intensity values **≥ 127** → become **255 (white)** 
- Pixels with intensity values **< 127** → become **0 (black)** 

This creates a clean **black-and-white** image that highlights the main shapes and removes most background noise. 
Binary thresholding is commonly used before tasks like object detection, shape analysis, and feature extraction.

##%%
# Global binary thresholding
_, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) #255 maxval->The value assigned to pixels above the threshold

plt.imshow(thresh, cmap="gray")
plt.title("Binary Threshold (T = 127)")
plt.axis("off")
plt.show()

##%% md
### 🌓 Adaptive Thresholding (Simplified)

Adaptive thresholding is used when the image has **uneven lighting**. 
Instead of using one global threshold, it calculates a **separate threshold for each small region** of the image.

We use `cv2.adaptiveThreshold()` with these settings:

- **ADAPTIVE_THRESH_MEAN_C** 
Threshold = (mean of local neighborhood) − C

- **THRESH_BINARY** 
Output pixels become either **0 (black)** or **255 (white)**.

- **blockSize = 11** 
Size of the small region used to compute the local mean.

- **C = 2** 
A small constant that adjusts the threshold.

This method produces a cleaner binary image in areas with different lighting, making segmentation more accurate.

##%%
adaptive = cv2.adaptiveThreshold(img, 255,
cv2.ADAPTIVE_THRESH_MEAN_C,
cv2.THRESH_BINARY,
11, 2)

plt.imshow(adaptive, cmap="gray")
plt.title("Adaptive Threshold")
plt.axis("off")
plt.show()

##%% md
### 📈 Otsu’s Thresholding (Automatic Binarization)

Otsu’s method is a global thresholding technique that **automatically chooses the best threshold value** based on the image histogram. 
No need to set a threshold manually.

Using `cv2.threshold()` with the `THRESH_OTSU` flag:

- The optimal threshold is computed from the histogram.
- Pixels **above** that value → **255 (white)**
- Pixels **below** → **0 (black)**

Otsu’s method works best when the image has a **bimodal histogram** (two clear intensity groups). It produces a clean binary image even when the contrast varies.

##%%
# Otsu's thresholding (automatic global threshold)
_, otsu = cv2.threshold(
img,
0, # ignored when using OTSU
255,
cv2.THRESH_BINARY + cv2.THRESH_OTSU
)

plt.imshow(otsu, cmap="gray")
plt.title("Otsu Thresholding")
plt.axis("off")
plt.show()

##%% md
## 15. Summary of Techniques Covered

In this notebook, you learned how to:

- Load and display images with `skimage` and `matplotlib`
- Visualize RGB channels and convert to grayscale
- Normalize pixel intensities to [0, 1]
- Use `ImageDataGenerator` for:
- Flipping
- Rotation
- Enhance contrast using:
- Histogram Equalization
- CLAHE (local contrast)
- Apply smoothing filters:
- Mean, Gaussian, Median
- Detect edges using:
- Sobel and Laplacian filters
- Sharpen images using a custom convolution kernel
- Perform basic segmentation with:
- Global thresholding
- Adaptive thresholding
- Otsu's method

These operations form a strong foundation for **image preprocessing** in computer vision pipelines before feeding images into machine learning or deep learning models.



AND CLASSIFIER USING KNN HE GAVE TOO 


##%% md
# K-Nearest Neighbors (KNN) for Image Classification

## Introduction

This notebook demonstrates the application of the **K-Nearest Neighbors (KNN)** algorithm to the task of **image classification**.

KNN is a supervised learning method that classifies data points based on the labels of their nearest neighbors in the feature space. In the context of image classification, each image is represented as a vector of features, which may be raw pixel values or extracted features from a preprocessing pipeline.

The primary objective of this notebook is to illustrate the process of:

- Training a KNN classifier on a labeled image dataset.
- Predicting the class labels of unseen test images.
- Evaluating the model's performance using standard metrics.

Throughout the notebook, we will:

- Train a KNN classifier on the MNIST handwritten digits dataset.
- Predict class labels for test images.
- Evaluate performance using:
- Accuracy
- Confusion Matrix
- Class-specific metrics (Precision, Recall, F1-score)

##%%
# Importing the dataset from Keras
from keras.datasets import mnist

# Load data: (x_train, y_train) for training, (x_test, y_test) for testing
(x_train, y_train), (x_test, y_test) = mnist.load_data()

##%% md
## Loading the MNIST Dataset

In this step, we load the **MNIST dataset** using `keras.datasets`.

- `x_train`, `x_test`: contain the image data as NumPy arrays.
- `y_train`, `y_test`: contain the corresponding labels (digits from 0 to 9).

The dataset consists of:

- 60,000 training images
- 10,000 test images

Each image is a grayscale `28 × 28` pixel image of a handwritten digit. 
- A grayscale image has one channel (not RGB).
- Each pixel is a value from 0 → 255:
- 0 = black
- 255 = white
- values in between (1–254) = different shades of gray
##%%
# Checking the data types
print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))

# Checking the shapes
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

##%%
import matplotlib.pyplot as plt

plt.gray() # Display images in black & white

plt.figure(figsize=(10, 9)) # Adjust figure size

# Display a grid of 3×3 images
for i in range(9):
plt.subplot(3, 3, i + 1)
plt.imshow(x_train[i])

plt.suptitle("Sample MNIST Training Images")
plt.show()

##%%
# Display the first 5 labels (true digit classes)
for i in range(5):
print(f"Image {i} label:", y_train[i])

##%%
# Checking the minimum and maximum values of x_train before normalization
print("Before normalization:")
print("x_train min:", x_train.min())
print("x_train max:", x_train.max())

##%% md
## Normalization for KNN

We perform two preprocessing steps:

1. **Data type conversion** 
- Convert from integer (`uint8`) to `float32` to support arithmetic operations and ML algorithms.

2. **Normalization to [0, 1]** 
- Divide pixel values by 255.0.
- Ensures that all features (pixels) contribute proportionally when computing distances.

This is especially important for **KNN**, which relies on distance calculations in feature space. 
Without normalization, dimensions with larger numeric ranges could dominate the distance.

##%%
# Data Normalization

# 1. Convert to float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 2. Scale pixel values to [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

##%%
# Checking the minimum and maximum values after normalization
print("After normalization:")
print("x_train min:", x_train.min())
print("x_train max:", x_train.max())

##%% md
## Flattening Images into Feature Vectors

KNN (and many classical ML algorithms) expect input data as **2D arrays** of shape:

- `(num_samples, num_features)`

Originally:

- `x_train` shape: `(num_samples, 28, 28)`

After reshaping:

- `X_train` shape: `(num_samples, 784)` where `784 = 28 × 28`.

Each image becomes a **784-dimensional vector**, and KNN computes distances between these vectors to determine similarity.

##%%
# Reshaping input data
# Each 28×28 image becomes a vector of length 784
X_train = x_train.reshape(len(x_train), -1)
X_test = x_test.reshape(len(x_test), -1)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

##%% md
## K-Nearest Neighbors (KNN) as a Lazy Learner

K-Nearest Neighbors (KNN) is considered a **lazy learning algorithm**.

- Unlike **eager learning** methods (e.g., decision trees, neural networks) that **learn a global model** during training,
- A **lazy learner** defers most computation until **prediction time**.
- Lazy learners do NOT build a model during training.
- They simply store the data and wait until a prediction is needed.

### Key characteristics

1. **Training Phase**
- KNN does not learn explicit model parameters.
- The `fit()` method simply **stores the training data and labels**.

2. **Prediction Phase**
- For each new input:
- Computes the distance to all stored training points.
- Finds the `k` nearest neighbors.
- Predicts the class using **majority voting** among these neighbors.

In this notebook, we will use `k = 3`, meaning each test image will be classified based on its 3 closest training images.




When you choose k=3, the model:
- Finds the 3 closest training samples to the test point
- Looks at their labels
- The majority vote becomes the prediction
##%%
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Choose number of neighbors
k = 3

# Initialize the KNN classifier
knn = KNeighborsClassifier(n_neighbors=k)

# "Training" step: store X_train and y_train
knn.fit(X_train, y_train)

# Predict labels for the test set
predicted_labels = knn.predict(X_test)


## Evaluating Accuracy

We now measure how often the model predicts the correct label.

**Accuracy** is defined as:
$$
\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total number of samples}}
$$
Where:

- **Number of correct predictions**: test samples where prediction = true label.
- **Total number of samples**: all test samples.

On a balanced dataset like MNIST, accuracy is a useful overall performance metric.

##%%
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predicted_labels)
print("Test set accuracy:", accuracy)

##%% md
## Confusion Matrix

Accuracy gives a single number, but it does not show **which classes are confused** with each other.

A **confusion matrix** gives a more detailed view:

- It is a square matrix of size `number_of_classes × number_of_classes`.
- **Rows** = Actual classes 
- **Columns** = Predicted classes 
- Entry (i, j) = number of samples of class *i* predicted as class *j*.

We will also visualize it with a **heatmap**:

- `annot=True` → display counts inside cells.
- `fmt="d"` → show integers.
- Helps identify digits that the model often misclassifies.

##%%

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_test, predicted_labels)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
##%%

num_examples = 20 # show first 20 examples
for i in range(num_examples):
print(f"Sample {i}: Actual = {y_test[i]}, Predicted = {predicted_labels[i]}")
##%% md
This cell evaluates the K-Nearest Neighbors (KNN) classifier on the test dataset using class-specific performance metrics beyond simple accuracy. The classification_report function computes:

1. Precision for each class:

$$
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives + False Positives}}
$$

Measures the proportion of correctly predicted instances among all instances predicted as that class.

High precision indicates a low false positive rate.

***

$$
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives + False Negatives}}
$$

Measures the proportion of correctly predicted instances among all actual instances of that class.

High recall indicates that most actual instances of the class are correctly identified.
***

$$
\text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision + Recall}}
$$

Harmonic mean of precision and recall, providing a single metric that balances both.
***

**Explanation of terms:**

- **True Positives (TP):** 
Cases where the model correctly predicts the positive class. 
**Example:** Actual = 1, Predicted = 1 
The model said “positive” and it was truly positive

- **False Positives (FP):** 
Cases where the model predicts positive, but the actual class is negative. 
**Example:** Actual = 0, Predicted = 1 
The model gave a “positive” prediction when it should be negative 

- **False Negatives (FN):** 
Cases where the model predicts negative, but the actual class is positive. 
**Example:** Actual = 1, Predicted = 0 
The model missed a positive case
- **Total number of samples:** Total number of instances in the test set.


##%%
from sklearn.metrics import classification_report

report = classification_report(y_test, predicted_labels)
print(report)

##%% md
**Digit 0**

- Precision = 0.97 → 97% of predicted 0’s are correct
- Recall = 0.99 → model finds 99% of true 0’s
- F1 = 0.98 → excellent
- Support = 980 → number of true 0’s in the test set


GET TO WORK 



In [ ]:
# Train Logistic Regression classifier
print("Training Logistic Regression classifier...")
lr_classifier = LogisticRegression(
    max_iter=1000,
    random_state=42,
    multi_class='multinomial',  # For multi-class classification
    solver='lbfgs'  # Good for small to medium datasets
)

lr_classifier.fit(X_train_scaled, y_train)

# Predictions
y_pred_lr = lr_classifier.predict(X_test_scaled)

# Calculate accuracy
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"\nLogistic Regression Accuracy: {accuracy_lr:.4f} ({accuracy_lr*100:.2f}%)")


### 10.1 Logistic Regression - Confusion Matrix


In [ ]:
# Confusion Matrix for Logistic Regression
cm_lr = confusion_matrix(y_test, y_pred_lr)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[f'Class {i}' for i in unique_emotions],
            yticklabels=[f'Class {i}' for i in unique_emotions])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Logistic Regression - Confusion Matrix')
plt.tight_layout()
plt.show()


### 10.2 Logistic Regression - Classification Report
in image data there is 3 CSV I WANT TO CREATE. Logistic regression & kmeans as classifiers on an image dataset (5 classes at maximum). IN JUPYTER NOTEBOOK 

HERE IS THE IMAGE PREPROCCESING TURTORIALS TA GAVE 

##%%
#importing required Libraries
import numpy as np
import tensorflow
import keras
import os
import glob
from skimage import io
import skimage
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
##%% md
# Image Preprocessing in Python (Lecture Notebook)

This notebook introduces fundamental image preprocessing techniques using **Python**, **OpenCV**, **scikit-image**, and **Keras**.

We will cover:

- Loading and visualizing images
- RGB channels and grayscale conversion
- Normalization
- Data augmentation (flipping and rotation)
- Contrast enhancement (Histogram Equalization & CLAHE)
- Smoothing filters (mean, Gaussian, median)
- Edge detection (Sobel, Laplacian)
- Sharpening with custom kernels
- Thresholding and basic image segmentation

##%% md
## 1. Loading and Visualizing the Original Image

In this step we:

1. Load an image from disk using `skimage.io.imread`.
2. Visualize it using `matplotlib`.

Why this step is important:

- It lets us inspect the **raw image**.
- We can observe:
- Resolution (width × height)
- Colors and lighting
- Any visible noise or blur
- This serves as a **baseline** before applying preprocessing.

> Make sure `fruits.png` is in the same folder as your notebook, or provide the full path.

##%%
import os
# accessing an image file from the dataset classes
image = io.imread('fruits.png')

# plotting the original image
i, (im1) = plt.subplots(1)
i.set_figwidth(15)
im1.imshow(image)
##%% md
## 2. Visualizing the Original Image and Its RGB Color Channels

Color images are stored as 3D arrays: **(height, width, channels)**.

- Channel 0 → Red
- Channel 1 → Green
- Channel 2 → Blue

In this section we:

- Show the original image
- Show each channel separately (R, G, B)

Why this helps:

- Understand which colors dominate the image.
- See how information is distributed across channels.
- Useful before performing color-based processing (e.g. segmentation, enhancement).

##%%
# plotting the original image and the RGB channels

i, (im1, im2, im3, im4) = plt.subplots(1, 4, sharey=True) # sharey -> All subplots will share the same Y-axis.
i.set_figwidth(20)
print(image.shape)
im1.imshow(image) #Original image
im2.imshow(image[:, : , 0]) #Red
im3.imshow(image[:, : , 1]) #Green
im4.imshow(image[:, : , 2]) #Blue
i.suptitle('Original & RGB image channels')
##%% md
The RGB image is converted into a **grayscale** representation using `skimage.color.rgb2gray()`.
A grayscale image reduces the three color channels (Red, Green, and Blue) into a single intensity channel by applying a weighted sum that reflects human visual perception. This simplification is commonly used in many preprocessing tasks, such as edge detection, thresholding, and filtering, where color information is not required.

The resulting grayscale image is then displayed using matplotlib with the 'gray' colormap to ensure proper visualization of intensity variations.
##%% md
## 3. Converting RGB Image to Grayscale

We convert the RGB image into a **single-channel grayscale** image using:

```python
skimage.color.rgb2gray(image) ---> it is automatically normalizes the grayscale image to the range [0, 1].

##%%
gray_image = skimage.color.rgb2gray(image)
plt.imshow(gray_image, cmap = 'gray')
##%% md
## 4. Normalization (Min–Max Scaling)

We normalize the grayscale image to the range **[0, 1]** using:

$$
\text{norm}(I) = \frac{I - I_{\min}}{I_{\max} - I_{\min}}
$$

Why normalize?

- Standardizes pixel values.
- Improves numerical stability for ML/DL models.
- Helps networks train faster and more reliably.
- Makes images from different sources more comparable.

We will compute the normalized image and display it.

##%%
norm_image = (gray_image - np.min(gray_image)) / (np.max(gray_image) - np.min(gray_image))
plt.imshow(norm_image)
##%% md
## 5. Installing `keras_preprocessing` (For Data Augmentation)

To perform geometric transformations (data augmentation) in Keras, we use:

- `ImageDataGenerator` from `keras_preprocessing.image`

If the package is not already installed, we install it using `pip` inside the notebook.

##%%
#!pip install keras_preprocessing
##%% md
## 6. Preparing Image Batch for Data Augmentation

`ImageDataGenerator` expects input as a batch of images with shape:

- `(batch_size, height, width, channels)`

We will:

1. Convert our image to `float32`.
2. Add a new axis to create a batch of size 1 using `np.expand_dims`.

This will be used for flipping and rotation examples.

##%%
from numpy import expand_dims
print(image.shape)
# Ensure we use the RGB image for augmentation
samples = expand_dims(image.astype('float32'), axis=0) # shape: (1, H, W, 3)


##%%
print("Batch shape:", samples.shape)

##%% md
# Geometric Transformations
In this step, we apply **geometric transformations** to the images using the `ImageDataGenerator` class from `Keras`. Geometric transformations are a form of data augmentation.

We use `ImageDataGenerator` to apply:

- `horizontal_flip=True` → flip left–right
- `vertical_flip=True` → flip top–bottom

Why do this?

- Data augmentation: increases dataset size without collecting new images.
- Makes models more robust to orientation changes.
- Helps prevent overfitting.

Explain:

* Creating an ImageDataGenerator instance with the desired transformations.

* Generating batches of images using the `.flow()` method, which produces transformed images on the fly.

* Iterating through the generated images and converting them to unsigned integers (`uint8`) for proper visualization.

* Plotting the transformed images side by side to observe the effects of horizontal and vertical flips.

We will:

1. Create an `ImageDataGenerator` with flipping options.
2. Generate 3 augmented images.
3. Display them side by side.
##%%
from keras_preprocessing.image import ImageDataGenerator

# ImageDataGenerator for flipping
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

# Create an iterator
it = datagen.flow(samples, batch_size=1) #batch_size=1 ->Each time you call the iterator, it returns only 1 augmented image.

# Plot some flipped images
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

for i in range(3):
# Get a batch (1 image), take first image
batch = next(it)[0].astype('uint8')
axes[i].imshow(batch)
axes[i].set_title(f"Flip Sample {i+1}")
axes[i].axis("off")

plt.suptitle("Horizontal & Vertical Flips")
plt.show()

##%% md
## 8. Geometric Transformations – Rotation


Apply `random rotation transformations` to the images as part of data augmentation. The `rotation_range` parameter specifies the maximum rotation angle (in degrees) for randomly rotating images. Here, `rotation_range=40` allows images to rotate within ±40 degrees.

**Why use rotation?**
- Simulates different orientations.
- Helps models generalize better when objects are rotated in real-world data.
##%%
# ImageDataGenerator for rotation
# Fills empty pixels with the value of the nearest pixel
# The image will be randomly rotated between –40° and +40°.
datagen = ImageDataGenerator(rotation_range=40, fill_mode='nearest') #

# Create an iterator
it = datagen.flow(samples, batch_size=1)

# Plot some flipped images
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

for i in range(3):
# Get a batch (1 image), take first image
batch = next(it)[0].astype('uint8')
axes[i].imshow(batch)
axes[i].set_title(f"Rotation Sample {i+1}")
axes[i].axis("off")

plt.suptitle("Random Rotations (±40°)")
plt.show()
##%% md
## 9. Histogram Equalization for Contrast Enhancement (زيادة التباين + تحسين الوضوح)

We now switch to **OpenCV** (`cv2`) for some operations.

### Goal:
- Improve image contrast using **histogram equalization**.

Steps:
1. Load the image in **grayscale**.
2. Apply `cv2.equalizeHist`.
3. Compare original vs equalized images.

Histogram equalization:
- Spreads pixel intensities (بتوزّع قيم الإضاءة) across the full range `[0, 255]`.
- Makes dark regions brighter and bright regions clearer. (إظهار التفاصيل المخفية)
- Helpful when the image looks too dark or too washed out. (مفيد في الصور ذات الإضاءة السيئة)

##%%
import cv2
import matplotlib.pyplot as plt

# Load grayscale image
img = cv2.imread("fruits.png", 0) # 0 → grayscale

# Histogram equalization
equalized = cv2.equalizeHist(img)

# Plot results
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Original Grayscale Image")
plt.imshow(img, cmap="gray")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Equalized Image")
plt.imshow(equalized, cmap="gray")
plt.axis("off")

plt.show()

##%% md
## 10. Visualizing Histograms Before and After Equalization

To better understand the effect of **histogram equalization**, we plot the pixel intensity histograms of the original and equalized images.

* `The original histogram` shows the distribution of pixel values in the grayscale image. In many cases, the values are concentrated in a narrow range, which can make the image appear dark or washed out.

* `The equalized histogram` illustrates how the pixel intensities have been redistributed across the full range (0–255). This spreading of values increases the contrast and highlights previously hidden details.

By comparing these histograms side by side, it becomes evident that histogram equalization effectively enhances image contrast while preserving the overall structure of the image. Visual inspection alongside histogram analysis provides a clear and quantitative understanding of the preprocessing step’s impact.
##%%
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.title("Original Histogram")
plt.hist(img.ravel(), 256, [0, 256])

plt.subplot(1, 2, 2)
plt.title("Equalized Histogram")
plt.hist(equalized.ravel(), 256, [0, 256])

plt.show()

##%% md
## 11. Contrast Enhancement Using CLAHE

**CLAHE** = Contrast Limited Adaptive Histogram Equalization.
- used to improve the contrast of a color image

Differences vs normal histogram equalization:

- Works on **small tiles** instead of whole image. 
- Limits contrast amplification (avoids over-enhancing noise). ( يحد من تضخيم التباين (يتجنب التشويش المفرط).)
- Very useful for images with **uneven lighting**. (الإضاءة غير المتساوية)
- A small amount of noise may appear due to processing each tile separately.

Steps:

1. Convert BGR image to LAB color space. (LAB separates lightness (L) from color (A and B).)
2. Apply CLAHE on the L (lightness) channel.
3. Merge channels and convert back to BGR/RGB.
4. Compare before and after.

**LAB channels:**
- L = Lightness (0–255)
- A = Green–Red scale
- B = Blue–Yellow scale
##%% md
**clipLimit=3.0**
- Prevents over-amplifying noise
- Higher value → stronger contrast

**tileGridSize=(8, 8)**
- Image is divided into 8×8 regions
- Contrast enhancement is applied locally
- Good for images with uneven lighting
##%%
# Load image (BGR)
img_color = cv2.imread("fruits.png")

# Convert to LAB color space
lab = cv2.cvtColor(img_color, cv2.COLOR_BGR2LAB)
l, a, b = cv2.split(lab)

# Apply CLAHE on L-channel (lightness)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
cl = clahe.apply(l)

# Merge channels and convert back to BGR
lab_clahe = cv2.merge((cl, a, b))
final = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

# Show Results
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("After CLAHE")
plt.imshow(cv2.cvtColor(final, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.show()

##%% md
## 12. Low-Pass Filters (Smoothing / Blurring)

In this step, we apply a **Mean Filter** to the grayscale image. The mean filter is a type of low-pass filter that **smooths** the image by reducing intensity variations between neighboring pixels. **It works by replacing each pixel value with the average of its surrounding pixels defined by a kernel** (in this case, a 3×3 window).


We will apply:

1. **Mean Filter** (يأخذ متوسط قيم البكسلات داخل نافذة (Kernel).)
2. **Gaussian Blur**
3. **Median Filter** 

Why smoothing?

- Reduces noise.
- Softens edges.
- Often used before edge detection or segmentation.
##%%
# Reload grayscale image for filtering
img = cv2.imread("fruits.png", 0)

# Mean (average) filter with 3x3 kernel
mean = cv2.blur(img, (3, 3))

plt.imshow(mean, cmap="gray")
plt.title("Mean Filter (3x3)")
plt.axis("off")
plt.show()

##%%
# Gaussian blur with 5x5 kernel, sigma = 0 (auto)
gaussian = cv2.GaussianBlur(img, (5, 5), 0)

plt.imshow(gaussian, cmap="gray")
plt.title("Gaussian Blur (5x5)")
plt.axis("off")
plt.show()

##%%
median = cv2.medianBlur(img, 5) # 5x5 neighborhood

plt.imshow(median, cmap="gray")
plt.title("Median Filter (5x5)")
plt.axis("off")
plt.show()

##%% md
## 13. High-Pass Filters (Edge Detection & Sharpening)

Edge Detection Using **Sobel Filter**

The Sobel operator computes the gradient of pixel intensities(تدرج شدة البكسل) in both horizontal (X) and vertical (Y) directions, highlighting regions with significant intensity changes.

* sobel(x) detects vertical edges by calculating horizontal intensity gradients.

* sobel(y) detects horizontal edges by calculating vertical intensity gradients.

Edge detection is a fundamental preprocessing step in computer vision, used to identify object boundaries, enhance features for segmentation, and extract structural information from images. Visualizing both X and Y gradients separately allows us to analyze the directionality and strength of edges in the image.
##%% md
**Understanding dx and dy**

**- dx = 1, dy = 0 → detect changes along X-axis**
- Finds vertical edges
- (Because vertical edges change in X direction)

**- dx = 0, dy = 1 → detect changes along Y-axis**
- Finds horizontal edges
- (Because horizontal edges change in Y direction)
##%%
sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3) # depth,1->SobelX, 0->SobelY
sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3) 

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(sobelx, cmap="gray")
plt.title("Sobel X (Vertical Edges)")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(sobely, cmap="gray")
plt.title("Sobel Y (Horizontal Edges)")
plt.axis("off")

plt.show()

##%% md
### 🔍 Laplacian Filter (Simplified Explanation)

The **Laplacian filter** is a high-pass filter used to detect edges in an image.

- It responds to **sharp changes in intensity**.
- Unlike Sobel (which finds horizontal or vertical edges separately), 
**Laplacian detects edges in all directions at once**.
- This makes it useful for highlighting **fine details** and **object boundaries**.

The resulting image shows bright edges and suppresses smooth, low-detail areas.

##%%
laplacian = cv2.Laplacian(img, cv2.CV_64F)

plt.imshow(laplacian, cmap="gray")
plt.title("Laplacian (All Edges)")
plt.axis("off")
plt.show()

##%% md
### Sharpening Filter (Custom Kernel)

In this step, we apply a sharpening filter to enhance the details and edges of the grayscale image. The filter is implemented using a convolution kernel that emphasizes the central pixel relative to its neighbors:

$$
\begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0
\end{bmatrix}
$$
- The center pixel is multiplied by 5.
- Neighboring pixels are subtracted.
- This increases contrast at edges → image looks sharper.
- This is a **balanced** sharpening kernel.

This kernel increases the contrast between a pixel and its surrounding pixels, effectively highlighting edges and fine structures while maintaining the overall brightness of the image. Sharpening is a common preprocessing technique to improve the visibility of important features, which can be useful in tasks such as object recognition, segmentation, and feature extraction.
##%%
kernel = np.array([[0,-1,0],
[-1,5,-1],
[0,-1,0]])

sharpened = cv2.filter2D(img, -1, kernel)

plt.imshow(sharpened, cmap="gray")
plt.title("Sharpened Image")
plt.axis("off")
plt.show()

##%% md
## 14. Segmentation with Thresholding

### 🌓 Binary Thresholding

In this step, we apply binary thresholding to the grayscale image. Thresholding is a fundamental technique in image segmentation.

Binary thresholding separates the image into **two groups of pixels**: 
- **Foreground (white)** 
- **Background (black)** 

We use `cv2.threshold()` with a threshold value of **127**:

- Pixels with intensity values **≥ 127** → become **255 (white)** 
- Pixels with intensity values **< 127** → become **0 (black)** 

This creates a clean **black-and-white** image that highlights the main shapes and removes most background noise. 
Binary thresholding is commonly used before tasks like object detection, shape analysis, and feature extraction.

##%%
# Global binary thresholding
_, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY) #255 maxval->The value assigned to pixels above the threshold

plt.imshow(thresh, cmap="gray")
plt.title("Binary Threshold (T = 127)")
plt.axis("off")
plt.show()

##%% md
### 🌓 Adaptive Thresholding (Simplified)

Adaptive thresholding is used when the image has **uneven lighting**. 
Instead of using one global threshold, it calculates a **separate threshold for each small region** of the image.

We use `cv2.adaptiveThreshold()` with these settings:

- **ADAPTIVE_THRESH_MEAN_C** 
Threshold = (mean of local neighborhood) − C

- **THRESH_BINARY** 
Output pixels become either **0 (black)** or **255 (white)**.

- **blockSize = 11** 
Size of the small region used to compute the local mean.

- **C = 2** 
A small constant that adjusts the threshold.

This method produces a cleaner binary image in areas with different lighting, making segmentation more accurate.

##%%
adaptive = cv2.adaptiveThreshold(img, 255,
cv2.ADAPTIVE_THRESH_MEAN_C,
cv2.THRESH_BINARY,
11, 2)

plt.imshow(adaptive, cmap="gray")
plt.title("Adaptive Threshold")
plt.axis("off")
plt.show()

##%% md
### 📈 Otsu’s Thresholding (Automatic Binarization)

Otsu’s method is a global thresholding technique that **automatically chooses the best threshold value** based on the image histogram. 
No need to set a threshold manually.

Using `cv2.threshold()` with the `THRESH_OTSU` flag:

- The optimal threshold is computed from the histogram.
- Pixels **above** that value → **255 (white)**
- Pixels **below** → **0 (black)**

Otsu’s method works best when the image has a **bimodal histogram** (two clear intensity groups). It produces a clean binary image even when the contrast varies.

##%%
# Otsu's thresholding (automatic global threshold)
_, otsu = cv2.threshold(
img,
0, # ignored when using OTSU
255,
cv2.THRESH_BINARY + cv2.THRESH_OTSU
)

plt.imshow(otsu, cmap="gray")
plt.title("Otsu Thresholding")
plt.axis("off")
plt.show()

##%% md
## 15. Summary of Techniques Covered

In this notebook, you learned how to:

- Load and display images with `skimage` and `matplotlib`
- Visualize RGB channels and convert to grayscale
- Normalize pixel intensities to [0, 1]
- Use `ImageDataGenerator` for:
- Flipping
- Rotation
- Enhance contrast using:
- Histogram Equalization
- CLAHE (local contrast)
- Apply smoothing filters:
- Mean, Gaussian, Median
- Detect edges using:
- Sobel and Laplacian filters
- Sharpen images using a custom convolution kernel
- Perform basic segmentation with:
- Global thresholding
- Adaptive thresholding
- Otsu's method

These operations form a strong foundation for **image preprocessing** in computer vision pipelines before feeding images into machine learning or deep learning models.



AND CLASSIFIER USING KNN HE GAVE TOO 


##%% md
# K-Nearest Neighbors (KNN) for Image Classification

## Introduction

This notebook demonstrates the application of the **K-Nearest Neighbors (KNN)** algorithm to the task of **image classification**.

KNN is a supervised learning method that classifies data points based on the labels of their nearest neighbors in the feature space. In the context of image classification, each image is represented as a vector of features, which may be raw pixel values or extracted features from a preprocessing pipeline.

The primary objective of this notebook is to illustrate the process of:

- Training a KNN classifier on a labeled image dataset.
- Predicting the class labels of unseen test images.
- Evaluating the model's performance using standard metrics.

Throughout the notebook, we will:

- Train a KNN classifier on the MNIST handwritten digits dataset.
- Predict class labels for test images.
- Evaluate performance using:
- Accuracy
- Confusion Matrix
- Class-specific metrics (Precision, Recall, F1-score)

##%%
# Importing the dataset from Keras
from keras.datasets import mnist

# Load data: (x_train, y_train) for training, (x_test, y_test) for testing
(x_train, y_train), (x_test, y_test) = mnist.load_data()

##%% md
## Loading the MNIST Dataset

In this step, we load the **MNIST dataset** using `keras.datasets`.

- `x_train`, `x_test`: contain the image data as NumPy arrays.
- `y_train`, `y_test`: contain the corresponding labels (digits from 0 to 9).

The dataset consists of:

- 60,000 training images
- 10,000 test images

Each image is a grayscale `28 × 28` pixel image of a handwritten digit. 
- A grayscale image has one channel (not RGB).
- Each pixel is a value from 0 → 255:
- 0 = black
- 255 = white
- values in between (1–254) = different shades of gray
##%%
# Checking the data types
print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))

# Checking the shapes
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

##%%
import matplotlib.pyplot as plt

plt.gray() # Display images in black & white

plt.figure(figsize=(10, 9)) # Adjust figure size

# Display a grid of 3×3 images
for i in range(9):
plt.subplot(3, 3, i + 1)
plt.imshow(x_train[i])

plt.suptitle("Sample MNIST Training Images")
plt.show()

##%%
# Display the first 5 labels (true digit classes)
for i in range(5):
print(f"Image {i} label:", y_train[i])

##%%
# Checking the minimum and maximum values of x_train before normalization
print("Before normalization:")
print("x_train min:", x_train.min())
print("x_train max:", x_train.max())

##%% md
## Normalization for KNN

We perform two preprocessing steps:

1. **Data type conversion** 
- Convert from integer (`uint8`) to `float32` to support arithmetic operations and ML algorithms.

2. **Normalization to [0, 1]** 
- Divide pixel values by 255.0.
- Ensures that all features (pixels) contribute proportionally when computing distances.

This is especially important for **KNN**, which relies on distance calculations in feature space. 
Without normalization, dimensions with larger numeric ranges could dominate the distance.

##%%
# Data Normalization

# 1. Convert to float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# 2. Scale pixel values to [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

##%%
# Checking the minimum and maximum values after normalization
print("After normalization:")
print("x_train min:", x_train.min())
print("x_train max:", x_train.max())

##%% md
## Flattening Images into Feature Vectors

KNN (and many classical ML algorithms) expect input data as **2D arrays** of shape:

- `(num_samples, num_features)`

Originally:

- `x_train` shape: `(num_samples, 28, 28)`

After reshaping:

- `X_train` shape: `(num_samples, 784)` where `784 = 28 × 28`.

Each image becomes a **784-dimensional vector**, and KNN computes distances between these vectors to determine similarity.

##%%
# Reshaping input data
# Each 28×28 image becomes a vector of length 784
X_train = x_train.reshape(len(x_train), -1)
X_test = x_test.reshape(len(x_test), -1)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

##%% md
## K-Nearest Neighbors (KNN) as a Lazy Learner

K-Nearest Neighbors (KNN) is considered a **lazy learning algorithm**.

- Unlike **eager learning** methods (e.g., decision trees, neural networks) that **learn a global model** during training,
- A **lazy learner** defers most computation until **prediction time**.
- Lazy learners do NOT build a model during training.
- They simply store the data and wait until a prediction is needed.

### Key characteristics

1. **Training Phase**
- KNN does not learn explicit model parameters.
- The `fit()` method simply **stores the training data and labels**.

2. **Prediction Phase**
- For each new input:
- Computes the distance to all stored training points.
- Finds the `k` nearest neighbors.
- Predicts the class using **majority voting** among these neighbors.

In this notebook, we will use `k = 3`, meaning each test image will be classified based on its 3 closest training images.




When you choose k=3, the model:
- Finds the 3 closest training samples to the test point
- Looks at their labels
- The majority vote becomes the prediction
##%%
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Choose number of neighbors
k = 3

# Initialize the KNN classifier
knn = KNeighborsClassifier(n_neighbors=k)

# "Training" step: store X_train and y_train
knn.fit(X_train, y_train)

# Predict labels for the test set
predicted_labels = knn.predict(X_test)


## Evaluating Accuracy

We now measure how often the model predicts the correct label.

**Accuracy** is defined as:
$$
\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total number of samples}}
$$
Where:

- **Number of correct predictions**: test samples where prediction = true label.
- **Total number of samples**: all test samples.

On a balanced dataset like MNIST, accuracy is a useful overall performance metric.

##%%
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predicted_labels)
print("Test set accuracy:", accuracy)

##%% md
## Confusion Matrix

Accuracy gives a single number, but it does not show **which classes are confused** with each other.

A **confusion matrix** gives a more detailed view:

- It is a square matrix of size `number_of_classes × number_of_classes`.
- **Rows** = Actual classes 
- **Columns** = Predicted classes 
- Entry (i, j) = number of samples of class *i* predicted as class *j*.

We will also visualize it with a **heatmap**:

- `annot=True` → display counts inside cells.
- `fmt="d"` → show integers.
- Helps identify digits that the model often misclassifies.

##%%

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_test, predicted_labels)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
##%%

num_examples = 20 # show first 20 examples
for i in range(num_examples):
print(f"Sample {i}: Actual = {y_test[i]}, Predicted = {predicted_labels[i]}")
##%% md
This cell evaluates the K-Nearest Neighbors (KNN) classifier on the test dataset using class-specific performance metrics beyond simple accuracy. The classification_report function computes:

1. Precision for each class:

$$
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives + False Positives}}
$$

Measures the proportion of correctly predicted instances among all instances predicted as that class.

High precision indicates a low false positive rate.

***

$$
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives + False Negatives}}
$$

Measures the proportion of correctly predicted instances among all actual instances of that class.

High recall indicates that most actual instances of the class are correctly identified.
***

$$
\text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision + Recall}}
$$

Harmonic mean of precision and recall, providing a single metric that balances both.
***

**Explanation of terms:**

- **True Positives (TP):** 
Cases where the model correctly predicts the positive class. 
**Example:** Actual = 1, Predicted = 1 
The model said “positive” and it was truly positive

- **False Positives (FP):** 
Cases where the model predicts positive, but the actual class is negative. 
**Example:** Actual = 0, Predicted = 1 
The model gave a “positive” prediction when it should be negative 

- **False Negatives (FN):** 
Cases where the model predicts negative, but the actual class is positive. 
**Example:** Actual = 1, Predicted = 0 
The model missed a positive case
- **Total number of samples:** Total number of instances in the test set.


##%%
from sklearn.metrics import classification_report

report = classification_report(y_test, predicted_labels)
print(report)

##%% md
**Digit 0**

- Precision = 0.97 → 97% of predicted 0’s are correct
- Recall = 0.99 → model finds 99% of true 0’s
- F1 = 0.98 → excellent
- Support = 980 → number of true 0’s in the test set


GET TO WORK 



In [ ]:
# Classification Report for Logistic Regression
print("Logistic Regression - Classification Report:")
print("=" * 60)
report_lr = classification_report(y_test, y_pred_lr, 
                                  target_names=[f'Class {i}' for i in unique_emotions])
print(report_lr)


## 11. K-Means Classifier

K-Means is an unsupervised clustering algorithm. To use it for classification:
1. Cluster the training data into k clusters (where k = number of classes)
2. Map each cluster to a class label based on majority voting
3. Predict test samples by assigning them to the nearest cluster


In [ ]:
# Determine number of clusters (should equal number of classes)
n_clusters = len(unique_emotions)
print(f"Number of clusters: {n_clusters}")

# Train K-Means on training data
print("\nTraining K-Means classifier...")
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10, max_iter=300)
kmeans.fit(X_train_scaled)

# Get cluster assignments for training data
train_clusters = kmeans.predict(X_train_scaled)

# Map clusters to class labels using majority voting
cluster_to_class = {}
for cluster_id in range(n_clusters):
    # Find all training samples in this cluster
    cluster_mask = (train_clusters == cluster_id)
    cluster_labels = y_train[cluster_mask]
    
    # Find the most common class label in this cluster
    if len(cluster_labels) > 0:
        most_common_class = np.bincount(cluster_labels).argmax()
        cluster_to_class[cluster_id] = most_common_class
        print(f"Cluster {cluster_id} -> Class {most_common_class} ({np.sum(cluster_labels == most_common_class)}/{len(cluster_labels)} samples)")
    else:
        # If cluster is empty, assign to first class
        cluster_to_class[cluster_id] = unique_emotions[0]

print(f"\nCluster to class mapping: {cluster_to_class}")


In [ ]:
# Predict test samples using K-Means
test_clusters = kmeans.predict(X_test_scaled)

# Map cluster assignments to class labels
y_pred_kmeans = np.array([cluster_to_class[cluster_id] for cluster_id in test_clusters])

# Calculate accuracy
accuracy_kmeans = accuracy_score(y_test, y_pred_kmeans)
print(f"\nK-Means Accuracy: {accuracy_kmeans:.4f} ({accuracy_kmeans*100:.2f}%)")


### 11.1 K-Means - Confusion Matrix


In [ ]:
# Confusion Matrix for K-Means
cm_kmeans = confusion_matrix(y_test, y_pred_kmeans)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_kmeans, annot=True, fmt='d', cmap='Oranges',
            xticklabels=[f'Class {i}' for i in unique_emotions],
            yticklabels=[f'Class {i}' for i in unique_emotions])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('K-Means - Confusion Matrix')
plt.tight_layout()
plt.show()


### 11.2 K-Means - Classification Report


In [ ]:
# Classification Report for K-Means
print("K-Means - Classification Report:")
print("=" * 60)
report_kmeans = classification_report(y_test, y_pred_kmeans,
                                      target_names=[f'Class {i}' for i in unique_emotions])
print(report_kmeans)


## 12. Comparison of Both Classifiers


In [ ]:
# Compare accuracies
print("=" * 60)
print("CLASSIFIER COMPARISON")
print("=" * 60)
print(f"\nLogistic Regression Accuracy: {accuracy_lr:.4f} ({accuracy_lr*100:.2f}%)")
print(f"K-Means Accuracy:            {accuracy_kmeans:.4f} ({accuracy_kmeans*100:.2f}%)")
print(f"\nDifference: {abs(accuracy_lr - accuracy_kmeans):.4f} ({abs(accuracy_lr - accuracy_kmeans)*100:.2f}%)")

# Visual comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Logistic Regression confusion matrix
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=[f'C{i}' for i in unique_emotions],
            yticklabels=[f'C{i}' for i in unique_emotions])
ax1.set_xlabel('Predicted')
ax1.set_ylabel('Actual')
ax1.set_title(f'Logistic Regression (Accuracy: {accuracy_lr*100:.2f}%)')

# K-Means confusion matrix
sns.heatmap(cm_kmeans, annot=True, fmt='d', cmap='Oranges', ax=ax2,
            xticklabels=[f'C{i}' for i in unique_emotions],
            yticklabels=[f'C{i}' for i in unique_emotions])
ax2.set_xlabel('Predicted')
ax2.set_ylabel('Actual')
ax2.set_title(f'K-Means (Accuracy: {accuracy_kmeans*100:.2f}%)')

plt.tight_layout()
plt.show()


## 13. Visualizing Some Predictions


In [ ]:
# Display some test predictions
n_samples_to_show = 12
indices = np.random.choice(len(X_test), n_samples_to_show, replace=False)

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for idx, i in enumerate(indices):
    # Reshape flattened image back to 2D for display
    img = X_test[i].reshape(height, width)
    
    axes[idx].imshow(img, cmap='gray')
    
    # Get predictions
    true_label = y_test[i]
    pred_lr = y_pred_lr[i]
    pred_kmeans = y_pred_kmeans[i]
    
    # Determine if predictions are correct
    lr_correct = "✓" if pred_lr == true_label else "✗"
    kmeans_correct = "✓" if pred_kmeans == true_label else "✗"
    
    title = f"True: {true_label}\nLR: {pred_lr} {lr_correct} | K-Means: {pred_kmeans} {kmeans_correct}"
    axes[idx].set_title(title, fontsize=9)
    axes[idx].axis('off')

plt.suptitle('Sample Test Predictions', fontsize=14)
plt.tight_layout()
plt.show()


## 14. Summary and Conclusions


In [ ]:
print("=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"\nDataset:")
print(f"  - Total samples: {len(X)}")
print(f"  - Training samples: {len(X_train)}")
print(f"  - Test samples: {len(X_test)}")
print(f"  - Number of classes: {len(unique_emotions)}")
print(f"  - Image size: {height}×{width} pixels")
print(f"  - Features per image: {X.shape[1]}")

print(f"\nClassifier Performance:")
print(f"  - Logistic Regression: {accuracy_lr*100:.2f}%")
print(f"  - K-Means:            {accuracy_kmeans*100:.2f}%")

print(f"\nObservations:")
if accuracy_lr > accuracy_kmeans:
    print(f"  - Logistic Regression performs better (supervised learning advantage)")
else:
    print(f"  - K-Means performs better (unusual, may indicate good cluster structure)")
    
print(f"  - K-Means is unsupervised and doesn't use label information during training")
print(f"  - Logistic Regression uses label information and is better suited for classification")
